# Baselines - playlist generation for known users

In [1]:
%matplotlib inline

import os, sys, time, gzip
import pickle as pkl
import numpy as np
from scipy.sparse import lil_matrix, issparse

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# from tools import calc_RPrecision_HitRate
from tools import calc_metrics, diversity, softmax
# from tools import calc_Precision_Recall

In [3]:
TOPs = [5, 10, 20, 30, 50, 100, 200, 300, 500, 700, 1000]

In [4]:
datasets = ['aotm2011', '30music']

In [5]:
dix = 0
dataset_name = datasets[dix]
dataset_name

'aotm2011'

In [6]:
data_dir = 'data/%s/coldstart/setting3' % dataset_name
X = pkl.load(gzip.open(os.path.join(data_dir, 'X.pkl.gz'), 'rb'))
Y_train = pkl.load(gzip.open(os.path.join(data_dir, 'Y_train.pkl.gz'), 'rb'))
Y_test = pkl.load(gzip.open(os.path.join(data_dir, 'Y_test.pkl.gz'), 'rb'))
song2pop_train = pkl.load(gzip.open(os.path.join(data_dir, 'song2pop_train.pkl.gz'), 'rb'))

In [7]:
playlists3 = pkl.load(gzip.open(os.path.join(data_dir, 'playlists_train_test_s3.pkl.gz'), 'rb'))
train_playlists = playlists3['train_playlists']
test_playlists = playlists3['test_playlists']
user2songs = dict()

for pl, u in train_playlists:
    try:
        user2songs[u].update(set(pl))
    except KeyError:
        user2songs[u] = set(pl)

In [8]:
all_songs = pkl.load(gzip.open(os.path.join(data_dir, 'all_songs.pkl.gz'), 'rb'))
index2song = {ix: sid for ix, (sid, _) in enumerate(all_songs)}

In [9]:
song2index = {sid: ix for ix, (sid, _) in enumerate(all_songs)}

In [10]:
_song2artist = pkl.load(gzip.open('data/msd/song2artist.pkl.gz', 'rb'))
song2artist = {sid: _song2artist[sid] for sid, _ in all_songs if sid in _song2artist}

In [11]:
artist2songs = dict()

for sid in sorted(song2artist):
    artist = song2artist[sid]
    try:
        artist2songs[artist].append(sid)
    except KeyError:
        artist2songs[artist] = [sid]

In [12]:
print('{:,} | {:,}'.format(len(song2artist), len(artist2songs)))

111,993 | 15,698


In [13]:
song2genre = pkl.load(gzip.open('data/msd/song2genre.pkl.gz', 'rb'))

In [14]:
song2pop = pkl.load(gzip.open(os.path.join(data_dir, 'song2pop.pkl.gz'), 'rb'))

### Collocated Artists - Greatest Hits (CAGH)

Compute the similarity of two artist $a_1$ and $a_2$ given a set of playlist $P$:   
$$
\text{sim}(a_1, a_2) 
= \frac{\sum_{p \in P} \delta(a_1, p) \times \delta(a_2, p)}
       {\sqrt{\sum_{p \in P} \delta(a_1, p) \times \sum_{p \in P} \delta(a_2, p)}}
$$
where
$$
\delta(a, p) 
= \begin{cases}
1, \ \text{at least one song in playlist $p$ is from artist $a$}, \\
0, \ \text{otherwise}.
\end{cases}
$$

Recommend according to the popularity of songs, but weighted by similarity of (`artist in user's listening history`, `artist of song`).

In [15]:
all_artist = sorted(set([song2artist[sid] for pl, _ in train_playlists for sid in pl if sid in song2artist]))

In [16]:
artist2index = {aid: ix for ix, aid in enumerate(all_artist)}

In [17]:
Na = len(all_artist)
Np = len(train_playlists)
Delta = lil_matrix((Na, Np), dtype=np.float)
for j in range(Np):
    pl_artist = sorted(set([song2artist[sid] for sid in train_playlists[j][0] if sid in song2artist]))
    ix = [artist2index[aid] for aid in pl_artist]
    Delta[ix, j] = 1

In [18]:
Delta = Delta.tocsr()
Dsum = Delta.sum(axis=1).A.reshape(-1)
ColloMat = Delta.dot(Delta.T).A

assert np.all(np.isclose(ColloMat.diagonal(), Dsum))

In [19]:
print(len(Dsum), len(all_artist))

15698 15698


In [20]:
#type(ColloMat)

In [21]:
T1 = 1. / np.sqrt(Dsum)
NormMat = np.dot(T1.reshape(Na, 1), T1.reshape(1, Na))

WeightMat = np.multiply(ColloMat, NormMat)

In [22]:
rps_cagh = []
hitrates_cagh = {top: [] for top in TOPs}
aucs_cagh = []
spreads_cagh = []
novelties_cagh = {top: dict() for top in TOPs}
artist_diversities_cagh = {top: [] for top in TOPs}
genre_diversities_cagh = {top: [] for top in TOPs}
np.random.seed(0)

assert Y_test.shape[1] == len(test_playlists)

sid_legal = [sid for sid, _ in all_songs if sid in song2artist]
aix_legal = [artist2index[song2artist[sid]] for sid in sid_legal]
pop_legal = np.asarray([song2pop_train[sid] for sid in sid_legal])
ix_legal = [song2index[sid] for sid in sid_legal]

prev_u = None
prev_y = None

for j in range(Y_test.shape[1]):
    sys.stdout.write('\r%d / %d' % (j+1, Y_test.shape[1]))
    sys.stdout.flush()
    y_true = Y_test[:, j].A.reshape(-1)
    
    u = test_playlists[j][1]
    if prev_u is None or prev_u != u:
        artists = sorted(set([song2artist[sid] for sid in user2songs[u] if sid in song2artist]))
        artists_ix = [artist2index[aid] for aid in artists]
        y_pred = np.zeros(y_true.shape)
        y_pred[ix_legal] = np.log(pop_legal) * np.asarray([WeightMat[aix, artists_ix].sum() for aix in aix_legal])

        # for ix in ix_legal:
        #     sid = index2song[ix]
        #     aix = artist2index[song2artist[sid]]
        #     pop = song2pop_test[sid]
        #     y_pred[ix] = pop * WeightMat[aix, artists_ix].sum()
        
        prev_u = u
        prev_y = y_pred
    else:
        y_pred = prev_y

    # rp, hr_dict = calc_RPrecision_HitRate(y_true, y_pred, tops=TOPs)
    rp, hr_dict, auc = calc_metrics(y_true, y_pred, tops=TOPs)
    rps_cagh.append(rp)
    for top in TOPs:
        hitrates_cagh[top].append(hr_dict[top])
    aucs_cagh.append(auc)
    
    # spread
    y_pred_prob = softmax(y_pred)
    spreads_cagh.append(-np.dot(y_pred_prob, np.log(y_pred_prob)))

    # novelty
    sortix = np.argsort(-y_pred)
    for top in TOPs:
        nov = np.mean([-np.log2(song2pop[index2song[ix]]) for ix in sortix[:top]])
        try:
            novelties_cagh[top][u].append(nov)
        except KeyError:
            novelties_cagh[top][u] = [nov]
    
    # artist/genre diversity
    for top in TOPs:
        artist_vec = np.array([song2artist[index2song[ix]] if index2song[ix] in song2artist
                               else str(np.random.rand()) for ix in sortix[:top]])
        genre_vec = np.array([song2genre[index2song[ix]] if index2song[ix] in song2genre \
                              else str(np.random.rand()) for ix in sortix[:top]])
        artist_diversities_cagh[top].append( diversity(artist_vec) )
        genre_diversities_cagh[top].append( diversity(genre_vec) )

print('\n%d / %d' % (len(rps_cagh), Y_test.shape[1]))

9233 / 9233
9233 / 9233


In [23]:
# fig = plt.figure(figsize=[20, 5])
# ax1 = plt.subplot(131)
# ax1.hist(rps_cagh, bins=100)
# ax1.set_yscale('log')
# ax1.set_title('R-Precision')
# #ax.set_xlim(0, xmax)
# ax2 = plt.subplot(132)
# ax2.hist(aucs_cagh, bins=100)
# ax2.set_yscale('log')
# ax2.set_title('AUC')
# pass

In [24]:
cagh_perf = {dataset_name: {'Test': {'R-Precision': np.mean(rps_cagh), 
                                     'Hit-Rate': {top: np.mean(hitrates_cagh[top]) for top in TOPs},
                                     'AUC': np.mean(aucs_cagh),
                                     'Spread': np.mean(spreads_cagh),
                                     'Novelty': {t: np.mean([np.mean(novelties_cagh[t][u]) 
                                                             for u in novelties_cagh[t]]) for t in TOPs},
                                     'Artist-Diversity': {t: np.mean(artist_diversities_cagh[t]) for t in TOPs},
                                     'Genre-Diversity': {t: np.mean(genre_diversities_cagh[t]) for t in TOPs}},
                            'Test_All': {'R-Precision': rps_cagh, 
                                         'Hit-Rate': {top: hitrates_cagh[top] for top in TOPs},
                                         'AUC': aucs_cagh,
                                         'Spread': spreads_cagh,
                                         'Novelty': novelties_cagh,
                                         'Artist-Diversity': artist_diversities_cagh,
                                         'Genre-Diversity': genre_diversities_cagh}}}
cagh_perf[dataset_name]['Test']

{'R-Precision': 0.012648125825220133,
 'Hit-Rate': {5: 0.006794196503848195,
  10: 0.01208350999271688,
  20: 0.021370544769167037,
  30: 0.02967795640871416,
  50: 0.04340629322160431,
  100: 0.07494026103183174,
  200: 0.12315234475645184,
  300: 0.16320545478612028,
  500: 0.22616272296084824,
  700: 0.27680425799067454,
  1000: 0.3378445596027157},
 'AUC': 0.9423883674704033,
 'Spread': 2.296042808595775,
 'Novelty': {5: -8.565827748218268,
  10: -8.343505653283179,
  20: -8.086288371767095,
  30: -7.911403828279804,
  50: -7.680629527858549,
  100: -7.311076021345004,
  200: -6.901649279341681,
  300: -6.656786877332179,
  500: -6.354166461685341,
  700: -6.167590537784116,
  1000: -5.977997427445344},
 'Artist-Diversity': {5: 0.4208166359796383,
  10: 0.47656594100870064,
  20: 0.5720077297109339,
  30: 0.6405746006017894,
  50: 0.7324231405981649,
  100: 0.82584189123992,
  200: 0.8910979080390581,
  300: 0.9201191161411354,
  500: 0.9467726068404544,
  700: 0.9598935809669281,


In [25]:
fperf_cagh = os.path.join(data_dir, 'perf-cagh.pkl')
print(fperf_cagh)
pkl.dump(cagh_perf, open(fperf_cagh, 'wb'))
pkl.load(open(fperf_cagh, 'rb'))[dataset_name]['Test']

data/aotm2011/coldstart/setting3/perf-cagh.pkl


{'R-Precision': 0.012648125825220133,
 'Hit-Rate': {5: 0.006794196503848195,
  10: 0.01208350999271688,
  20: 0.021370544769167037,
  30: 0.02967795640871416,
  50: 0.04340629322160431,
  100: 0.07494026103183174,
  200: 0.12315234475645184,
  300: 0.16320545478612028,
  500: 0.22616272296084824,
  700: 0.27680425799067454,
  1000: 0.3378445596027157},
 'AUC': 0.9423883674704033,
 'Spread': 2.296042808595775,
 'Novelty': {5: -8.565827748218268,
  10: -8.343505653283179,
  20: -8.086288371767095,
  30: -7.911403828279804,
  50: -7.680629527858549,
  100: -7.311076021345004,
  200: -6.901649279341681,
  300: -6.656786877332179,
  500: -6.354166461685341,
  700: -6.167590537784116,
  1000: -5.977997427445344},
 'Artist-Diversity': {5: 0.4208166359796383,
  10: 0.47656594100870064,
  20: 0.5720077297109339,
  30: 0.6405746006017894,
  50: 0.7324231405981649,
  100: 0.82584189123992,
  200: 0.8910979080390581,
  300: 0.9201191161411354,
  500: 0.9467726068404544,
  700: 0.9598935809669281,


### Same Artists - Greatest Hits (SAGH)

Recommend according to the popularity of songs of artists in listening history.

In [26]:
rps_sagh = []
hitrates_sagh = {top: [] for top in TOPs}
aucs_sagh = []
spreads_sagh = []
novelties_sagh = {top: dict() for top in TOPs}
artist_diversities_sagh = {top: [] for top in TOPs}
genre_diversities_sagh = {top: [] for top in TOPs}
np.random.seed(0)

assert Y_test.shape[1] == len(test_playlists)
for j in range(Y_test.shape[1]):
    if (j+1) % 100 == 0:
        sys.stdout.write('\r%d / %d' % (j+1, Y_test.shape[1]))
        sys.stdout.flush()
    y_true = Y_test[:, j].A.reshape(-1)
    y_pred = np.zeros(y_true.shape)
    
    u = test_playlists[j][1]
    artists = sorted(set([song2artist[sid] for sid in user2songs[u] if sid in song2artist]))
    candidates = []
    for a in artists:
        candidates += artist2songs[a]
    candidates = sorted(set(candidates))
    if len(candidates) > 0:
        for sid in candidates:
            ix = song2index[sid]
            y_pred[ix] = np.log(song2pop_train[sid])

    # rp, hr_dict = calc_RPrecision_HitRate(y_true, y_pred, tops=TOPs)
    rp, hr_dict, auc = calc_metrics(y_true, y_pred, tops=TOPs)
    rps_sagh.append(rp)
    for top in TOPs:
        hitrates_sagh[top].append(hr_dict[top])
    aucs_sagh.append(auc)
    
    # spread
    y_pred_prob = softmax(y_pred)
    spreads_sagh.append(-np.dot(y_pred_prob, np.log(y_pred_prob)))

    # novelty
    sortix = np.argsort(-y_pred)
    for top in TOPs:
        nov = np.mean([-np.log2(song2pop[index2song[ix]]) for ix in sortix[:top]])
        try:
            novelties_sagh[top][u].append(nov)
        except KeyError:
            novelties_sagh[top][u] = [nov]
    
    # artist/genre diversity
    for top in TOPs:
        artist_vec = np.array([song2artist[index2song[ix]] if index2song[ix] in song2artist
                               else str(np.random.rand()) for ix in sortix[:top]])
        genre_vec = np.array([song2genre[index2song[ix]] if index2song[ix] in song2genre \
                              else str(np.random.rand()) for ix in sortix[:top]])
        artist_diversities_sagh[top].append( diversity(artist_vec) )
        genre_diversities_sagh[top].append( diversity(genre_vec) )
print('\n%d / %d' % (len(rps_sagh), Y_test.shape[1]))

9200 / 9233
9233 / 9233


In [27]:
# fig = plt.figure(figsize=[20, 5])
# ax1 = plt.subplot(131)
# ax1.hist(rps_sagh, bins=100)
# ax1.set_yscale('log')
# ax1.set_title('R-Precision')
# #ax.set_xlim(0, xmax)
# ax2 = plt.subplot(132)
# ax2.hist(aucs_sagh, bins=100)
# ax2.set_yscale('log')
# ax2.set_title('AUC')
# pass

In [28]:
sagh_perf = {dataset_name: {'Test': {'R-Precision': np.mean(rps_sagh), 
                                     'Hit-Rate': {top: np.mean(hitrates_sagh[top]) for top in TOPs},
                                     'AUC': np.mean(aucs_sagh),
                                     'Spread': np.mean(spreads_sagh),
                                     'Novelty': {t: np.mean([np.mean(novelties_sagh[t][u]) 
                                                             for u in novelties_sagh[t]]) for t in TOPs},
                                     'Artist-Diversity': {t: np.mean(artist_diversities_sagh[t]) for t in TOPs},
                                     'Genre-Diversity': {t: np.mean(genre_diversities_sagh[t]) for t in TOPs}},
                            'Test_All': {'R-Precision': rps_sagh, 
                                         'Hit-Rate': {top: hitrates_sagh[top] for top in TOPs},
                                         'AUC': aucs_sagh,
                                         'Spread': spreads_sagh,
                                         'Novelty': novelties_sagh,
                                         'Artist-Diversity': artist_diversities_sagh,
                                         'Genre-Diversity': genre_diversities_sagh}}}
sagh_perf[dataset_name]['Test']

{'R-Precision': 0.014952242385167722,
 'Hit-Rate': {5: 0.007987077367979455,
  10: 0.014584302521444464,
  20: 0.025609856218537824,
  30: 0.0359592103210705,
  50: 0.05415623995773431,
  100: 0.09182908333477224,
  200: 0.14930986549539385,
  300: 0.19456221896850043,
  500: 0.2639422543994849,
  700: 0.31923603952990715,
  1000: 0.3787249835565522},
 'AUC': 0.7978344628703591,
 'Spread': 10.410385965618532,
 'Novelty': {5: -8.977422870478174,
  10: -8.752837830630009,
  20: -8.47989372801478,
  30: -8.298531949409403,
  50: -8.038689626889926,
  100: -7.613221965221152,
  200: -7.082332198397629,
  300: -6.707389565652833,
  500: -6.1459804766956045,
  700: -5.706938666373436,
  1000: -5.168720070313489},
 'Artist-Diversity': {5: 0.9292754251055995,
  10: 0.9181847720134301,
  20: 0.9245663438353274,
  30: 0.9309814495979563,
  50: 0.9407077983364904,
  100: 0.953621452256782,
  200: 0.9643768392487728,
  300: 0.9692419549333549,
  500: 0.9742387658453483,
  700: 0.9771127095474033,


In [29]:
fperf_sagh = os.path.join(data_dir, 'perf-sagh.pkl')
print(fperf_sagh)
pkl.dump(sagh_perf, open(fperf_sagh, 'wb'))
pkl.load(open(fperf_sagh, 'rb'))[dataset_name]['Test']

data/aotm2011/coldstart/setting3/perf-sagh.pkl


{'R-Precision': 0.014952242385167722,
 'Hit-Rate': {5: 0.007987077367979455,
  10: 0.014584302521444464,
  20: 0.025609856218537824,
  30: 0.0359592103210705,
  50: 0.05415623995773431,
  100: 0.09182908333477224,
  200: 0.14930986549539385,
  300: 0.19456221896850043,
  500: 0.2639422543994849,
  700: 0.31923603952990715,
  1000: 0.3787249835565522},
 'AUC': 0.7978344628703591,
 'Spread': 10.410385965618532,
 'Novelty': {5: -8.977422870478174,
  10: -8.752837830630009,
  20: -8.47989372801478,
  30: -8.298531949409403,
  50: -8.038689626889926,
  100: -7.613221965221152,
  200: -7.082332198397629,
  300: -6.707389565652833,
  500: -6.1459804766956045,
  700: -5.706938666373436,
  1000: -5.168720070313489},
 'Artist-Diversity': {5: 0.9292754251055995,
  10: 0.9181847720134301,
  20: 0.9245663438353274,
  30: 0.9309814495979563,
  50: 0.9407077983364904,
  100: 0.953621452256782,
  200: 0.9643768392487728,
  300: 0.9692419549333549,
  500: 0.9742387658453483,
  700: 0.9771127095474033,


### Popularity based recommendation

In [31]:
rps_pop = []
hitrates_pop = {top: [] for top in TOPs}
aucs_pop = []
novelties_pop = {top: dict() for top in TOPs}
artist_diversities_pop = {top: [] for top in TOPs}
genre_diversities_pop = {top: [] for top in TOPs}
np.random.seed(0)

y_pred = np.array([song2pop_train[index2song[ix]] for ix in range(len(all_songs))])
y_pred_prob = softmax(np.log(y_pred))
spread_pop = -np.dot(y_pred_prob, np.log(y_pred_prob))
sortix = np.argsort(-y_pred)

assert Y_test.shape[1] == len(test_playlists)
for j in range(Y_test.shape[1]):
    if (j+1) % 10 == 0:
        sys.stdout.write('\r%d / %d' % (j+1, Y_test.shape[1]))
        sys.stdout.flush()
    y_true = Y_test[:, j].A.reshape(-1)
    
    # rp, hr_dict = calc_RPrecision_HitRate(y_true, y_pred, tops=TOPs)
    rp, hr_dict, auc = calc_metrics(y_true, y_pred, tops=TOPs)
    rps_pop.append(rp)
    for top in TOPs:
        hitrates_pop[top].append(hr_dict[top])
    aucs_pop.append(auc)

    # novelty
    u = test_playlists[j][1]
    for top in TOPs:
        nov = np.mean([-np.log2(song2pop[index2song[ix]]) for ix in sortix[:top]])
        try:
            novelties_pop[top][u].append(nov)
        except KeyError:
            novelties_pop[top][u] = [nov]

    # artist/genre diversity
    for top in TOPs:
        artist_vec = np.array([song2artist[index2song[ix]] if index2song[ix] in song2artist
                               else str(np.random.rand()) for ix in sortix[:top]])
        genre_vec = np.array([song2genre[index2song[ix]] if index2song[ix] in song2genre \
                              else str(np.random.rand()) for ix in sortix[:top]])
        artist_diversities_pop[top].append( diversity(artist_vec) )
        genre_diversities_pop[top].append( diversity(genre_vec) )
    
print('\n%d / %d' % (len(rps_pop), Y_test.shape[1]))

9230 / 9233
9233 / 9233


In [32]:
# fig = plt.figure(figsize=[20, 5])
# ax1 = plt.subplot(131)
# ax1.hist(rps_pop, bins=100)
# ax1.set_yscale('log')
# ax1.set_title('R-Precision')
# #ax.set_xlim(0, xmax)
# ax2 = plt.subplot(132)
# ax2.hist(aucs_pop, bins=100)
# ax2.set_yscale('log')
# ax2.set_title('AUC')
# pass

In [33]:
pop_perf = {dataset_name: {'Test': {'R-Precision': np.mean(rps_pop), 
                                    'Hit-Rate': {top: np.mean(hitrates_pop[top]) for top in TOPs},
                                    'AUC': np.mean(aucs_pop),
                                    'Spread': spread_pop,
                                    'Novelty': {t: np.mean([np.mean(novelties_pop[t][u]) for u in novelties_pop[t]]) 
                                                for t in TOPs},
                                    'Artist-Diversity': {top: np.mean(artist_diversities_pop[top]) for top in TOPs},
                                    'Genre-Diversity': {top: np.mean(genre_diversities_pop[top]) for top in TOPs}},
                           'Test_All': {'R-Precision': rps_pop, 
                                        'Hit-Rate': {top: hitrates_pop[top] for top in TOPs},
                                        'AUC': aucs_pop,
                                        'Spread': spread_pop,
                                        'Novelty': novelties_pop,
                                        'Artist-Diversity': artist_diversities_pop,
                                        'Genre-Diversity': genre_diversities_pop}}}
pop_perf[dataset_name]['Test']

{'R-Precision': 0.009816267242690632,
 'Hit-Rate': {5: 0.005094909581028886,
  10: 0.009166222222606553,
  20: 0.016499476426301812,
  30: 0.022890268838475017,
  50: 0.03419538096844783,
  100: 0.05754969582098008,
  200: 0.09576236300404181,
  300: 0.1256562491294101,
  500: 0.17588952940853056,
  700: 0.21824406560655835,
  1000: 0.2683455153982177},
 'AUC': 0.9380712780510054,
 'Spread': 10.51715942595781,
 'Novelty': {5: -9.396369039336703,
  10: -9.25592406191471,
  20: -9.086312906471067,
  30: -8.953573108138498,
  50: -8.762863172195072,
  100: -8.51010001395116,
  200: -8.216316950717681,
  300: -8.010798443060093,
  500: -7.7365684821441265,
  700: -7.552157246004768,
  1000: -7.332702451596431},
 'Artist-Diversity': {5: 1.0,
  10: 1.0,
  20: 0.9684210526315786,
  30: 0.9793103448275863,
  50: 0.9828571428571425,
  100: 0.9856565656565657,
  200: 0.9882412060301511,
  300: 0.991415830546265,
  500: 0.9932585170340681,
  700: 0.9943020641733091,
  1000: 0.9955795795795798},
 

In [34]:
fperf_pop = os.path.join(data_dir, 'perf-pop.pkl')
print(fperf_pop)
pkl.dump(pop_perf, open(fperf_pop, 'wb'))
pkl.load(open(fperf_pop, 'rb'))[dataset_name]['Test']

data/aotm2011/coldstart/setting3/perf-pop.pkl


{'R-Precision': 0.009816267242690632,
 'Hit-Rate': {5: 0.005094909581028886,
  10: 0.009166222222606553,
  20: 0.016499476426301812,
  30: 0.022890268838475017,
  50: 0.03419538096844783,
  100: 0.05754969582098008,
  200: 0.09576236300404181,
  300: 0.1256562491294101,
  500: 0.17588952940853056,
  700: 0.21824406560655835,
  1000: 0.2683455153982177},
 'AUC': 0.9380712780510054,
 'Spread': 10.51715942595781,
 'Novelty': {5: -9.396369039336703,
  10: -9.25592406191471,
  20: -9.086312906471067,
  30: -8.953573108138498,
  50: -8.762863172195072,
  100: -8.51010001395116,
  200: -8.216316950717681,
  300: -8.010798443060093,
  500: -7.7365684821441265,
  700: -7.552157246004768,
  1000: -7.332702451596431},
 'Artist-Diversity': {5: 1.0,
  10: 1.0,
  20: 0.9684210526315786,
  30: 0.9793103448275863,
  50: 0.9828571428571425,
  100: 0.9856565656565657,
  200: 0.9882412060301511,
  300: 0.991415830546265,
  500: 0.9932585170340681,
  700: 0.9943020641733091,
  1000: 0.9955795795795798},
 